In [1]:
from controllerLib import *
robot, scene, group, zeroPose  = initRobot()

Manipulator detected
World detetected
Initial Join values should be [77.32632695, -69.73220526, 121.29976102, -141.59751593, -90.06095192, 168.36293361]
Loading points
Total points loaded:  36
Setting Scene
Definning zeroPose


In [2]:
group.stop

Adding Constraints


In [3]:
pushTwo((2, 7), (2, 10))

Total points loaded:  36
Total time 4.43101787567


In [3]:
pushAll()

In [4]:
pushOne(2, 1)

In [42]:
createPath([(0, 0), (2, 4), (2, 1), (2, 9), (0, 0), (2, 8), (2, 11), (2, 7), (0, 3)])

Total points loaded:  36
Length 9
Total time 27.5369870663


In [5]:
createPath([[0,0],[2,11],[2,0],[0,11]])

Total points loaded:  36
Length 4
Total time 18.0489480495


In [1]:
import sys
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg
import math
from geometry_msgs.msg import PoseStamped
import time
import random
from tf.transformations import euler_from_quaternion, quaternion_from_euler
import numpy as np
from moveit_msgs.msg import Constraints, OrientationConstraint



In [2]:
moveit_commander.roscpp_initialize(sys.argv)

In [5]:
rospy.init_node('move_group_python_interface',anonymous=True)

In [21]:
help(rospy.signal_shutdown)

Help on function signal_shutdown in module rospy.core:

signal_shutdown(reason)
    Initiates shutdown process by signaling objects waiting on _shutdown_lock.
    Shutdown and pre-shutdown hooks are invoked.
    @param reason: human-readable shutdown reason, if applicable
    @type  reason: str



In [4]:
rospy.signal_shutdown("final")

In [8]:
trajectory = "0,0;2,11;2,0;0,11"
tokens = trajectory.split(";")
tPoints = []
for t in tokens:
    tPoints.append(map(lambda x: int(x), t.split(",")))
print tPoints

0,0
2,11
2,0
0,11
[[0, 0], [2, 11], [2, 0], [0, 11]]
